In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with Vertex AI Model Registry
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/get_started_with_model_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/get_started_with_model_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_registry/get_started_with_model_registry.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>         
</table>
<br/><br/><br/>

## Overview

This tutorial demonstrates how to use Vertex AI SDK to create and register multiple versions of a model using `Vertex AI Model Registry`.

Learn more about [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).

### Objective

In this tutorial, you learn how to use `Vertex AI Model Registry` to create and register multiple versions of a model.

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Model Registry`
- `Vertex AI Model` resources
- `Vertex AI Endpoint` resources

The steps performed include:

- Create and register a first version of a model to `Vertex AI Model Registry`.
- Create and register a second version of a model to `Vertex AI Model Registry`.
- Updating the model version which is the default (blessed).
- Deleting a model version.
- Retraining the next model version.

### Model

This tutorial uses a pre-trained image classification model from TensorFlow Hub, which is trained on ImageNet dataset.

Learn more about [ResNet V2 pretained model](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5). 

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages to execute this notebook.

In [ ]:
# Install the packages
USER=''
! pip3 install {USER} --upgrade google-cloud-aiplatform \
                                google-cloud-pipeline-components \
                                tensorflow==2.5 \
                                tensorflow-hub

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-unique-{PROJECT_ID}"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### Import libraries

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for training and prediction.

Set the variables `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)


Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more about [hardware accelerator support for your region](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators).

*Note*: TF releases before 2.3 for GPU support will fail to load the custom model in this tutorial. It is a known issue and fixed in TF 2.3. This is caused by static graph ops that are generated in the serving function. If you encounter this issue on your own custom models, use a container image for TF 2.3 with GPU support.

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### Set pre-built containers

Set the pre-built Docker container image for prediction.


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [ ]:
TF = "2.5".replace(".", "-")

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)


DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### Set machine type

Next, set the machine type to use for prediction.

- Set the variable `DEPLOY_COMPUTE` to configure  the compute resources for the VMs you will use for for prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [ ]:
DEPLOY_COMPUTE = "n1-standard-4"
print("Train machine type", DEPLOY_COMPUTE)

## Introduction to Model Registry

The `Vertex AI Model Registry` service provides you with the ability to create and group together multiple versions of a `Model` resource. With versioning, one can:

- Track the model lineage across versions
- Set which version is the default -- e.g., blessed.
- Other service operations, like deploy, will automatically use the default version -- w/o specifically specifying the version.
- Assign alias names -- such as development, staging and production -- to the versions.

## Get pretrained model from TensorFlow Hub

For demonstration purposes, this tutorial uses two versions of a pretrained model from TensorFlow Hub (TFHub), which are then uploaded to the `Vertex AI Model Registry` as version 1 and version 2 of the same `Model` resource. Once you have a `Vertex AI Model` resource, the model versions can be deployed to a `Vertex AI Endpoint` resource.

### Download the pretrained model

First, you download two versions of the pretrained model from TensorFlow Hub. The models get downloaded as a TF.Keras layer. To finalize the model, in this example, you create a `Sequential()` model with the downloaded TFHub model as a layer, and specify the input shape to the model.

In [ ]:
tfhub_model_v1 = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model_v1.build([None, 224, 224, 3])

tfhub_model_v1.summary()

In [ ]:
tfhub_model_v2 = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model_v2.build([None, 128, 128, 3])

tfhub_model_v2.summary()

### Save the model artifacts

At this point, the two versions of the model are in memory. Next, you save the model artifacts to a Cloud Storage location.

In [ ]:
MODEL_V1_DIR = BUCKET_URI + "/model/v1"
tfhub_model_v1.save(MODEL_V1_DIR)

MODEL_V2_DIR = BUCKET_URI + "/model/v2"
tfhub_model_v2.save(MODEL_V2_DIR)

### Upload version 1 of the TensorFlow Hub model to a `Vertex AI Model` resource

Next, you upload the first version of the model (`MODEL_DIR_V1`) as a `Model` resource in the `Model Registry`, with the additional following parameters:

- `is_default_version`: Whether this will be the default version for the `Model` resource.
- `version_ailiases`: User defined list of alternative alias names for the model version, such as `production`.
- `version_description`: User description of the model version.

When the first model version is created in the `Vertex AI Model Registry`, the property `version_id` will automatically be set to 1.

In [ ]:
model_v1 = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_V1_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    is_default_version=True,
    version_aliases=["v1", "v.1", "1"],
    version_description="This is the first version of the model",
)

print(model_v1)

### Upload version 2 of the TensorFlow Hub model to a `Vertex AI Model` resource

Next, you upload the second version of the model (`MODEL_DIR_V1`) as a `Model` resource in the `Model Registry`, with the additional following parameters:

- `parent_model`: The existing `Model` resource for which to add this model as the next model version.
- `is_default_version`: Whether this will be the default version for the `Model` resource. In this example, you change from the default from the first version to the second version of the model.
- `version_ailiases`: User defined list of alternative alias names for the model version, such as `production`.
- `version_description`: User description of the model version.

When asubsequent model version is created in the `Vertex AI Model Registry`, the property `version_id` will automatically be incremented. In this example, it will be set to 2 (2nd version).

In [ ]:
model_v2 = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_V2_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    parent_model=model_v1.resource_name,
    is_default_version=True,
    version_aliases=["v2", "v.2", "2"],
    version_description="This is the second version of the model",
)

print(model_v2)

### Get all versions of the parent model

Next, you list all the versions of a parent model using the `version_registry.list_versions()`.

In [ ]:
versions = model_v1.versioning_registry.list_versions()
for version in versions:
    print(version)

### Get all versions of a non-parent model

You repeat the same, but this time you use a non-parent model. As you can see, you get all the versions regardless if the model you specified is the parent or non-parent model.

In [ ]:
versions = model_v2.versioning_registry.list_versions()
for version in versions:
    print(version)

### Listing a `Model` resource

The remaining `Vertex AI` SDK methods relating to a `Model` resource will automatically use the default version of the `Model` resource.

Next, you use the `list()` method with a filter to get the `Model` resources you created above. In this example, there are two versions. Version 2 is set as the default version, so list() will only return the information on version 2 (default version).

In [ ]:
models = aip.Model.list(filter=f"display_name=example_{TIMESTAMP}")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

### Change the default model version

Next, you change which version of the `Model` resource is the default model version using the `versioning_registry.add_version_aliases()` method, with the following parameters:

- `version`: Which version, specified by the version_id, this operation applies to.
- `new_aliases`: The additional aliases to assign to the specified version.

In this example, you refer to the special alias `default` to change this model (version 1) as the default.

Next, you use the `list()` method which now will return vesion 1 instead of version 2 of the model.

In [ ]:
model_v2.versioning_registry.add_version_aliases(new_aliases=["default"], version="1")

models = aip.Model.list(filter=f"display_name=example_{TIMESTAMP}")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

## Creating an `Endpoint` resource

You create an `Endpoint` resource using the `Endpoint.create()` method. At a minimum, you specify the display name for the endpoint. Optionally, you can specify the project and location (region); otherwise the settings are inherited by the values you set when you initialized the Vertex AI SDK with the `init()` method.

In this example, the following parameters are specified:

- `display_name`: A human readable name for the `Endpoint` resource.
- `project`: Your project ID.
- `location`: Your region.
- `labels`: (optional) User defined metadata for the `Endpoint` in the form of key/value pairs.

This method returns an `Endpoint` object.

Learn more about [Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api).

In [ ]:
endpoint = aip.Endpoint.create(
    display_name="example_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## Deploying `Model` resources to an `Endpoint` resource.

You can deploy one of more `Vertex AI Model` resource instances to the same endpoint. Each `Vertex AI Model` resource that is deployed will have its own deployment container for the serving binary. 

*Note:* For this example, you specified the deployment container for the TFHub model in the previous step of uploading the model artifacts to a `Vertex AI Model` resource.

### Deploying a single `Endpoint` resource

In the next example, you deploy the default version of the `Vertex AI Model` resource to a `Vertex AI Endpoint` resource. The `Vertex AI Model` resource already has defined for it the deployment container image. To deploy, you specify the following additional configuration settings:

- The machine type.
- The (if any) type and number of GPUs.
- Static, manual or auto-scaling of VM instances.

In this example, you deploy the model with the minimal amount of specified parameters, as follows:

- `model`: The `Model` resource.
- `deployed_model_displayed_name`: The human readable name for the deployed model instance.
- `machine_type`: The machine type for each VM instance.

Do to the requirements to provision the resource, this may take upto a few minutes.

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

#### Get information on the deployed model

You can get the deployment settings of the deployed model from the `Endpoint` resource configuration data `gca_resource.deployed_models`. In this example, only one model is deployed -- hence the reference to the subscript `[0]`.

In [ ]:
endpoint.gca_resource.deployed_models[0]

### Undeploy `Model` resource from `Endpoint` resource

When a `Model` resource is deployed to an `Endpoint` resource, the deployed `Model` resource instance is assigned an ID -- commonly referred to as the deployed model ID.

You can undeploy a specific `Model` resource instance with the `undeploy()` method, with the following parameters:

- `deployed_model_id`: The ID assigned to the deployed model.

In [ ]:
deployed_model_id = endpoint.gca_resource.deployed_models[0].id
print(deployed_model_id)

endpoint.undeploy(deployed_model_id)

### Deleting a model version

To delete a version of a model from the `Model Registry`, you use the `versioning_registry.delete_version()` method, with the following parameters:

- `version`: Can be either the `version_id` or an alias from `version_alias`.

*Note:* You cannot delete the default model version.

In [ ]:
# Deletes a specific model version, as long as it isn't the default version.
model.versioning_registry.delete_version("2")

versions = model.versioning_registry.list_versions()
for version in versions:
    print(version)

### Deleting a `Model` resource

When you delete a `Model` resource, all the associated versions are deleted.

In [ ]:
model.delete()

### Specifying model versions for `Vertex AI Training`

When training models using the `Vertex AI Training` service, you can specify the trained model to be a parent model in the `Model Registry`, or a version of the parent model, with the following additional parameters to the `run()` method:

- `parent_model`: The existing `Model` resource for which to add this model as the next model version.
- `is_default_version`: Whether this will be the default version for the `Model` resource.
- `model_version_ailiases`: User defined list of alternative alias names for the model version, such as `production`.
- `model_version_description`: User description of the model version.

Below is an example usage:

```
# This job will upload a new, non-default version of the my-training-job model
job = aiplatform.CustomTrainingJob(
   display_name="my-training-job",
   script_path="training_script.py",
   container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-2:latest",
   requirements=["gcsfs==0.7.1"],
   model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-2:latest",
)

model = job.run(
   my_dataset,
   replica_count=1,
   machine_type="n1-standard-4",
   accelerator_type='NVIDIA_TESLA_K80',
   accelerator_count=1,
   model_id='my-model'
   model_version_aliases=["myalias"],
   model_version_description="My description",
   is_default_version=False,
)

```


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
delete_bucket = False

try:
    endpoint.undeploy_all()
    endpoint.delete()
except Exception as e:
    print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}